# DuckDB tutorial: Building AI Projects
## Content
1. About DuckDB
2. Creating the DuckDB database (Project 011)
3. Building a RAG Application with DuckDB (Project 022)
    3.1. Setting up
    3.2. Setting up GPT-4o and Embedding Model
    3.3. Using DuckDB as a vector database
    3.4. Creating a simple RAG application
4. Building a DuckDB SQL Query Engine Using an LLM (Project 03)
    4.1. Setting up
    4.2. Loading the DuckDB database
    4.3. Building the SQL query engine

# 1. About DuckDB

DuckDB là một hệ quản trị cơ sở dữ liệu phân tích hiện đại, hiệu suất cao, hoạt động trong bộ nhớ (in-memory). Nó hỗ trợ SQL và được thiết kế để xử lý các truy vấn phân tích phức tạp.

Đặc điểm chính:
- Dễ sử dụng: Không cần máy chủ, không có phụ thuộc ngoài, có thể nhúng vào ứng dụng.
- Tính năng phong phú: Hỗ trợ SQL mạnh mẽ, tích hợp tốt với Python và R.
- Hiệu suất cao: Sử dụng công cụ thực thi truy vấn dạng cột tối ưu hóa cho phân tích dữ liệu.
- Miễn phí & Mã nguồn mở: Phát hành theo giấy phép MIT.
- Di động: Chạy trên nhiều hệ điều hành và kiến trúc CPU, thậm chí trên trình duyệt.
- Mở rộng: Hỗ trợ bổ sung kiểu dữ liệu, hàm, định dạng tệp, cú pháp SQL.
- Đáng tin cậy: Kiểm thử nghiêm ngặt với hàng triệu truy vấn để đảm bảo ổn định.

In [ ]:
%pip install duckdb --upgrade

2.2.3


# 2. Creating the DuckDB database (Project 01)

In [19]:
import duckdb

con = duckdb.connect('datacamp.duckdb')
# It will create a database base file in your local directory. 

- Load dataset
- Create table duckdb
- Load dataset into table

For more information, see in 'workspace' folder

In [20]:
import numpy as np
import pandas as pd

path_dataset = "../../Datasets/Bank_marketing_dataset/Bank_marketing.csv"
bank_duck = duckdb.read_csv(path_dataset,sep=";")

In [21]:
con.execute("""
    CREATE TABLE IF NOT EXISTS bank AS 
    SELECT * FROM read_csv("../../Datasets/Bank_marketing_dataset/Bank_marketing.csv")
""")
con.execute("SHOW ALL TABLES").fetchdf()

,database,schema,name,column_names,column_types,temporary
0,datacamp,main,bank,"[index, age, job, marital, education, default,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, VA...",False


In [22]:
rel = con.table("bank")
rel.columns

['index',
 'age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'y']

In [23]:
# Let’s write a relation that uses multiple functions to analyze the data.
rel.filter("duration < 100").project("job,education,loan").order("job").limit(3).df()

,job,education,loan
0,blue-collar,unknown,no
1,blue-collar,unknown,no
2,blue-collar,basic.9y,no


The DuckDB query function allows SQL queries to be executed within the database, returning results that can be converted into various formats for further analysis.

In the code example bellow, we are running the SQL query to find out the job titles of clients over the age of 30, count the number of clients contacted for each job, and calculate the average duration of the campaign.

In [24]:
res = duckdb.query("""SELECT 
                            job,
                            COUNT(*) AS total_clients_contacted,
                            AVG(duration) AS avg_campaign_duration,
                        FROM 
                            "../../Datasets/Bank_marketing_dataset/Bank_marketing.csv"
                        WHERE 
                            age > 30
                        GROUP BY 
                            job
                        ORDER BY 
                            total_clients_contacted DESC;""")
res.df()

,job,total_clients_contacted,avg_campaign_duration
0,admin.,26,245.807692
1,blue-collar,23,320.695652
2,technician,13,385.153846
3,management,9,283.444444
4,services,8,323.750000
5,unknown,4,239.250000
6,housemaid,3,273.333333
7,entrepreneur,3,601.666667
8,retired,2,258.000000
9,unemployed,2,345.000000


In [25]:
con.close()

# 3. Building a RAG Application with DuckDB (Project 02)

## 3.1. Setting up

In [26]:
%%capture
%pip install duckdb
%pip install llama-index
%pip install llama-index-vector-stores-duckdb

In [29]:
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.core import Settings
from llama_index.vector_stores.duckdb import DuckDBVectorStore
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from IPython.display import Markdown
from IPython.display import display

import os

## 3.2. Setting up GPT-4o and Embedding Model

In [ ]:
llm = OpenAI(
    model='gpt-4o',
    api_key=os.environ['OPENAI_API_KEY']
)

In [ ]:
embed_model = OpenAIEmbedding(model='text-embedding-3-small')

In [ ]:
Settings.llm = llm
Settings.embed_model = embed_model

## 3.3. Using DuckDB as a vector database+

In [ ]:
documents = SimpleDirectoryReader('../../Datasets/Application_with_Groq_data/').load_data()
# This folder has pdf files

Chúng ta tạo một ko lưu trữ vector tên là blog bằng cách dùng một database đã được tạo sẵn. Sau đó chuyển (convert) dữ liệu của các file pdf thành các embedding và sau đó lưu vào kho lưu trữ vector.

In [ ]:
vector_store = DuckDBVectorStore(database_name = "datacamp.duckdb",table_name = "blog",persist_dir="./", embed_dim=1536)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

To check if our vector store was successfully created, we will connect the database using the DuckDB Python API and run the SQL query to display all the tables in the database. 

In [ ]:
import duckdb
con = duckdb.connect("datacamp.duckdb")

con.execute("SHOW ALL TABLES").fetchdf()

## 3.4. Creating a simple RAG application

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Who wrote 'GitHub Actions and MakeFile: A Hands-on Introduction'?")
display(Markdown(f"<b>{response}</b>"))

## 3.5. Creating a RAG chatbot with memory

Now, let’s create an advanced RAG application that uses the conversation history to generate the response. For that, we have to create a chat memory buffer and then a chat engine with memory, LLM, and vector store retriever. 

Tìm hiểu thêm về chatmemory là gì, MemGPT...

In [ ]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = CondensePlusContextChatEngine.from_defaults(
    index.as_retriever(),
    memory=memory,
    llm=llm
)

response = chat_engine.chat(
    "What is the easiest way of finetuning the Llama 3 model? Please provide step-by-step instructions."
)

display(Markdown(response.response))

In [ ]:
response = chat_engine.chat(
    "Could you please provide more details about the Post Fine-Tuning Steps?"
)
display(Markdown(response.response))

# 4. Building a DuckDB SQL Query Engine Using an LLM (Project 03)

## 4.1. Setting up

%pip install duckdb-engine -q

## 4.2. Loading the DuckDB database

In [ ]:
# load the DuckDB database using the create_engine function and then write a simple SQL query to check whether 
# it is successfully loaded. 
from sqlalchemy import create_engine

engine = create_engine("duckdb:///datacamp.duckdb")
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM bank LIMIT 3")
    print(cursor.fetchall())

[(0, 56, 'housemaid', 'married', 'basic.4y', 'no', 'no', 'no', 'telephone', 'may', 'mon', 261, 1, 999, 0, 'nonexistent', 1.1, 93.994, -36.4, 4.857, 5191, False), (1, 57, 'services', 'married', 'high.school', 'unknown', 'no', 'no', 'telephone', 'may', 'mon', 149, 1, 999, 0, 'nonexistent', 1.1, 93.994, -36.4, 4.857, 5191, False), (2, 37, 'services', 'married', 'high.school', 'no', 'yes', 'no', 'telephone', 'may', 'mon', 226, 1, 999, 0, 'nonexistent', 1.1, 93.994, -36.4, 4.857, 5191, False)]


In [35]:
# create a database Tool using the SQLDatabase function. Provide it with an engine object and table name.  
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine, include_tables=['bank'])

c:\Users\NGOVANUC\.conda\envs\.venv\Lib\site-packages\duckdb_engine\__init__.py:184: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


## 4.3. Building the SQL query engine

In [ ]:
# Create the SQL query engine using the NLSQLTableQueryEngine function by 
# providing it with the LlamaIndex SQL database object. 
from llama_index.core.query_engine import NLSQLTableQueryEngine
query_engine = NLSQLTableQueryEngine(sql_database)

# Ask the question from the query engine
response = query_engine.query("Which is the longest running campaign?")
print(response.response)

In [ ]:
response = query_engine.query("Which type of job has the most housing loan?")
print(response.response)

In [ ]:
# To check what is going on on the back end, we will print the metadata. 
print(response.metadata)

In [ ]:
engine.close()

# 5. Conclusion

DuckDB là một cơ sở dữ liệu nhanh, dễ sử dụng và tích hợp tốt với nhiều framework về dữ liệu và AI. Nó không có phụ thuộc, giúp triển khai linh hoạt mà không cần cài đặt phức tạp.

Trong hướng dẫn này đã tìm hiểu về DuckDB, API Python của nó, cách tạo bảng và thực hiện phân tích dữ liệu đơn giản. Ngoài ra, hai dự án thực hành đã được giới thiệu:
- Ứng dụng RAG sử dụng DuckDB làm vector database.
- DuckDB như một SQL query engine.